# Readme.md

# Introduction
This project is built to prototype an automation and intelligence collection use case. The use case is the ability  to populate [MISP](https://www.misp-project.org/) with the results from the [Shodan search](https://developer.shodan.io/api) and the follow on [Shodan host](https://developer.shodan.io/api) API. The objects stored in MISP must be unique, it needs to visually graph well, and overall, it must be useful to an analyst. 

This is different than other projects that simply load Shodan results into MISP. This code attempts to organize queries into certain MISP events and reduce, as much as possible, duplication of results when the Shodan host command is ran.

The end of this notebook contains example MISP searches.


### Quick start
Provide a Shodan search query, just like you would on the Shodan website, and it will conduct a Shodan search. Watch the debug output to see the code progress and go into MISP to see the event as it builds.


### How it works
You provide a Shodan search query. From the results, the IP address is extracted. For this IP address, the Shodan host API is called with the history parameter. All results for the same IP address are saved into the same MISP event. Shodan results are extracted to individual objects. For example, the Shodan HTTP module results are extracted into the `http-metadata` object. As the results are processed, if values match what is already in MISP for a specific object, then it is NOT added (i.e. only unique values are added). Logging is used throughout so you can watch the code as it progresses through each step. Change the logging to debug to see everything.

There are several custom MISP objects used in this proof-of-concept. Normally, a object template is created to support the custom object. Instead, I did not do that and suppressed the MISP warnings. For each object template that already existed, it is likely no longer matching the template due to the needs of this prototype. In the below, I list the objects and if they are custom and/or modified.

* shodan-report - Modified version of https://www.misp-project.org/objects.html#_shodan_report
* intel-collection - Custom
* http-metadata  - Custom
* domain-ip  https://www.misp-project.org/objects.html#_domain_ip
* x509 - https://www.misp-project.org/objects.html#_x509
* geolocation -  https://www.misp-project.org/objects.html#_geolocation
* ftp-meta - Custom
 

### Notes
* The object with name `intel-collection` is very important in ths design. Each unique IP from a Shodan search is referenced to this object as a trigger.
* Originally, I used `_shodan.id` as a unique identifier to determine which Shodan results have been processed. However, this value is not always present so instead I built my own custom identifier using data provided in the Shodan results.
* The geolocation latitude/longitude values in a Shodan host history result for an IP change over time which causes a bunch of nearly identical geolocation MISP objects. The code is changed to prevent this by not considering latitude nor longitude.
* I was never able to get the pymisp complex_query to work correcly otherwise, some of this code would be simpllier. This is an example at the bottom on how to use it (or at least shows why it doesn't work).


### Requirements
* [MISP](https://www.misp-project.org/)
* [pymisp](https://pypi.org/project/pymisp/)
* [shodan](https://pypi.org/project/shodan/)

###  Built using 
* Python 3.6
* MISP 2.4.128 
* pymisp 2.4.128
* shodan 1.21.3


### License
* agpl-3.0

# What do you want to search Shodan for?

In [1]:
# The query exactly as you would enter it on the Shodan website
your_shodan_query = "VNC config"

# The reason you are making this query. It is added as a MISP attribute comment.
search_reason = 'Testing!'

# MISP and Shodan Config

In [2]:
# Required for MISP
misp_url = 'https://<domain|IP>'
misp_key = 'key_here' # The MISP auth key can be found on the MISP web interface under the automation section
misp_verifycert = True

# Required for Shodan
shodanApiKey = 'key_here'

# Setup and Functions

In [4]:
# Built in Python 3.6.7
__author__ = "Kemp Langhorne"
__copyright__ = "Copyright (C) 2020 AskKemp.com"
__license__ = "agpl-3.0"
__version__ = "Beta 1.0"

from shodan import Shodan
from pymisp import PyMISP, MISPEvent, MISPObject
import json
import os
from datetime import datetime
import re
import hashlib
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)  # Remove InsecureRequestWarning

# logging setup
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO) # DEBUG is all the data
logging.info("TEST")  # Has to be here for Jupyter otherwise logging does not work.

# Temporary messsure to prevent pymisp from complaining about adding keys to object templtaes that don't exist
misplogger = logging.getLogger('pymisp')
misplogger.setLevel(logging.CRITICAL)

urllib3logger = logging.getLogger('urllib3')
urllib3logger.setLevel(logging.CRITICAL)


# Communicate with Shodan API
def do_query(searchterm: str, querytype: str) -> dict:
    """
    Conducts search against Shodan API
    Input: str searchterm that can be executed against Shodan API
    Returns: dict as received from API query
    """

    shodanAPI = Shodan(shodanApiKey)

    if querytype == "search":
        logging.info("Conducting Shodan API endpoint search")
        apiQuery = shodanAPI.search(searchterm)  # Search query; identical syntax to the website
    elif querytype == "host":
        logging.info("Conducting Shodan API endpoint host")
        apiQuery = shodanAPI.host(searchterm, history=True, minify=False)  # gives all history
    return apiQuery

INFO:root:TEST


In [5]:
# PyMISP object. Need for it all to work.
pymisp = PyMISP(misp_url, misp_key, ssl=False, debug=False) 

In [ ]:
def misp_object_find_compare(mispEvent, objectName, attributeNameCompareList):
    """
    For a MISPevent, it finds the provided attribute name in the provided MISP object name and then compares the discovered value to a provided value
    Input:
      mispEvent: MISPEvent Object
      objectName: str of MISP object name. e.g. x509 or http-metadata or geolocation
      attributeNameCompareList: list of tuples. Tuple in the format (MISP Object Attribute Name, Value to compare against)
    Returns: If values match then it returns the FIRST MISPobject with the match
    """
    logger.debug("\t[compare] Starting MISP value compare for object name {}".format(objectName))
    mispObjectList = mispEvent.get_objects_by_name(objectName)  # will return empty list when no match

    logger.debug("\t[compare] list input: {}".format(attributeNameCompareList))
  
    if len(mispObjectList) > 0:
        for objNum, mispObj in enumerate(mispObjectList):  # list of matching MISP objects
            logger.debug("\t[compare] Starting MISP object number: {}".format(objNum))

            numMatches = 0  # number of matches found for each MISPobject
            # For each MISPobject, determine if it contains all of needed attributes and values
            for mispAttribName, compareValue in attributeNameCompareList:
                logger.debug("\t[compare] Looking for attribute name {} with value {} and type {}.".format(mispAttribName, compareValue, type(compareValue)))

                if mispObj.get_attributes_by_relation(mispAttribName):
                    mispAttribValue = mispObj.get_attributes_by_relation(mispAttribName)
                    if len(mispAttribValue) > 0:
                        mispAttribValue = mispAttribValue[0].get('value')  # should be a list of 1 entry
                        if type(mispAttribValue) != type(compareValue):
                            logging.warning('\t[compare] type mismatch! MISP {}{} to provided {}{}'.format(mispAttribValue, type(mispAttribValue), compareValue, type(compareValue)))
                        #logging.debug('\t\t[compare] [Object {}] {} vs {}'.format(mispObj.uuid, mispAttribValue,compareValue))
                        if mispAttribValue == compareValue:
                            logging.debug('\t[compare] MATCH [Object {}] {} vs {}'.format(mispObj.uuid, mispAttribValue, compareValue))
                            numMatches = numMatches + 1
                        else:
                            logging.debug('\t[compare] NO match [Object {}] {} vs {}'.format(mispObj.uuid, mispAttribValue, compareValue))
                    else:
                        logger.info("\t[compare] No results for attribute name")
                else:
                    logger.info("\t[compare] Requested atrribute name {} does NOT exist in MISPobject".format(mispAttribName))
                    continue
                #logging.debug(numMatches)
                # If all tuples match the MISPobject, then it is a match
                logger.debug("\t[compare] input list length {} and numMatch {}".format(len(mispObjectList), numMatches))
                if len(attributeNameCompareList) == numMatches:
                    logger.debug("\t[compare] All values matched in object {}".format(mispObj.uuid))
                    logger.debug("\t[compare] Returning MISP Object with UUID {}".format(mispObj.uuid))
                    return mispObj

    else:
        logger.debug("\t[compare] Object name not found: {}".format(objectName))

    return None
   

def shodan_to_misp_event(shodanQueryMatch: dict, shodanQuery: str, shodanQueryPurpose: str, intelctlObj: None, mispEvent: None ) -> None:
    """
    Takes single Shodan result, creates MISP event, and creates all possible MISP objects in that MISP event.
    Input:
      shodanQueryMatch: dict containing a single shodan result. e.g. the matches key contains many results. This is expecting one of those results.
      shodanQuery: str containing the query used against the shodan API
      shodanQueryPurpose: str containing purpose for query. e.g. hunting for APT999
      intelctlObj: None and accepts MISPObject to allow for proper referencing (add_reference)
      mispEvent: None and accepts MISPEvent to allow for a current MISP event to be updated
    Returns: None
    """

    # MISP Event
    if mispEvent:  # MISPEvent passed thus it will be used and its contents updated
        logger.debug('\t[to-misp] Provided MISP Event')
        event = mispEvent
    else:  # create new MISP event
        logger.debug('\t[to-misp] Creating new MISP Event')
        event = MISPEvent()
        event.info = your_shodan_query  # Required
        event.distribution = 0          # Optional, defaults to MISP.default_event_distribution in MISP config
        event.threat_level_id = 2       # Optional, defaults to MISP.default_event_threat_level in MISP config
        event.analysis = 1              # Optional, defaults to 0 (initial analysis)
        event.add_tag('tlp:red')

    # MISP intel-collection
    if intelctlObj:  # MISPObject is passed thus use it instead of building one. This allows for proper referencing.
        logger.debug('\t[to-misp] Provided intel-collection object')
        obj_intelcollection = intelctlObj
    else:
        logger.debug('\t[to-misp] Creating intel-collection object')
        obj_intelcollection = MISPObject('intel-collection')  # Custom object
        obj_intelcollection.add_attribute('trigger', type='text', comment='This started off the entire chain of events', value='Jupyter notebook', disable_correlation=True)
        obj_intelcollection.add_attribute('query', type='text', comment='', value=shodanQuery)
        obj_intelcollection.add_attribute('purpose', type='text', comment='', value="Purpose is " + search_reason, disable_correlation=True)

    # MISP shodan-report object
    logger.debug('\t[to-misp] Creating shodan-report object')
    obj_shodanreport = MISPObject('shodan-report')  # Object template https://www.misp-project.org/objects.html#_shodan_report
    obj_intelcollection.add_reference(obj_shodanreport, "triggers")  # this object is caused by the intel collection script

    # Creating a unique hash which will be used in future searches to find if the item is already recorded in MISP by this script
    uniqResultsID = shodanQuery + shodanQueryMatch.get('ip_str') + str(shodanQueryMatch.get('hash'))  # + shodanQueryMatch.get('timestamp')
    uniqResultsIDhash = hashlib.md5(uniqResultsID.encode('utf-8')).hexdigest()
    obj_shodanreport.add_attribute('unique_id', type='text', comment='used by intel collection script', value=uniqResultsIDhash, disable_correlation=True)

    if shodanQueryMatch.get('hostnames'):
        for myhostname in shodanQueryMatch.get('hostnames'): # general property - List of hostnames for the IP
            obj_shodanreport.add_attribute('hostname', type='domain', value=myhostname)
    obj_shodanreport.add_attribute('first-seen', type='datetime', value=shodanQueryMatch.get('timestamp')) # general property - Date and time the information was collected
    obj_shodanreport.add_attribute('shodan_id', type='text', comment='my comment', value=shodanQueryMatch.get('_shodan').get('id'), disable_correlation=True) # shodan property - unique ID for this banner
    obj_shodanreport.add_attribute('data_hash', type='text', value=shodanQueryMatch.get('hash')) # general property - Numeric hash of the data property
    obj_shodanreport.add_attribute('data', type='text', value=shodanQueryMatch.get('data'), disable_correlation=True) # Text banner for the service. Correlation will occur using hash
    obj_shodanreport.add_attribute('product', type='text', value=shodanQueryMatch.get('product')) # Name of the software running the service
    obj_shodanreport.add_attribute('product_version', type='text', value=shodanQueryMatch.get('version')) # Version of the software

    # MISP IP domain-ip object
    if mispEvent and intelctlObj:  # if a mispevent and intelobject are provided, then the IP already exsts in the MISP event. Need to find that IP.
        logger.debug('\t[to-misp] Using existing domain-ip object')
        obj_domainip = event.get_objects_by_name('domain-ip')[0] # should only ever be one of these in the MISP event because the MISP event is unique per IP and Shodan query (see uniqIDhash creation method)
        obj_domainip.add_reference(obj_shodanreport, "contains")
    else:
        logger.debug('\t[to-misp] Creating domain-ip object')
        obj_domainip = MISPObject('domain-ip')  # https://www.misp-project.org/objects.html#_domain_ip
        obj_domainip.add_reference(obj_shodanreport, "derived-from")
        obj_domainip.add_attribute('ip', type='ip-dst', value=shodanQueryMatch.get('ip_str')) # general property - IP address as a string
        obj_domainip.add_attribute('port', type='port', value=shodanQueryMatch.get('port')) # general property - Port number for the service
        if shodanQueryMatch.get('domains'):
            for mydomainname in shodanQueryMatch.get('domains'): # general property - List of hostnames for the IP
                obj_domainip.add_attribute('domains', type='domain', value=shodanQueryMatch.get(mydomainname)) # List of domains for the IP
        obj_domainip.add_attribute('org', type='text', value=shodanQueryMatch.get('org')) # general property - Organization that is assigned the IP
    
    # MISP geolcation object      
    if shodanQueryMatch.get('location'):

        objectExists = False # holds if the provided geolocation object should be used or if a new object should be created
        if mispEvent and intelctlObj: # if a mispevent and intelobject are provided, then the geolocation may already exist and be the same                          

            # Check if values in provided http-meta object are the same as the one which already exists in MISP
            findAll = []
            #if shodanQueryMatch.get('location').get('latitude'): # No longer using because the lat/long changes over the history of the IP and creates a ton of unique MISPobjects
            #    findAll.append(('latitude', str(shodanQueryMatch.get('location').get('latitude')))) # Shodan value is float yet it is stored in MISP as str even when assigned as float...?
            #if shodanQueryMatch.get('location').get('longitude'):
            #    findAll.append(('longitude', str(shodanQueryMatch.get('location').get('longitude'))))
            if shodanQueryMatch.get('location').get('city'):
                findAll.append(('city', shodanQueryMatch.get('location').get('city')))
            if shodanQueryMatch.get('location').get('country_code'):
                findAll.append(('country', shodanQueryMatch.get('location').get('country_code')))
            objectWithValue = misp_object_find_compare(mispEvent=event, objectName='geolocation', attributeNameCompareList=findAll)
            if objectWithValue:
                logger.debug('\t[to-misp] Using existing geolocation object')
                obj_geolocation = objectWithValue
                #obj_geolocation.add_reference(obj_domainip, "derived-from") # Not needed
                objectExists = True
            else:
                objectExists = False
      
        if objectExists == False:  # create new geo location object
            logger.debug('\t[to-misp] Creating geolocation object') 
            obj_geolocation = MISPObject('geolocation')  # https://www.misp-project.org/objects.html#_geolocation
            obj_geolocation.add_reference(obj_domainip, "derived-from")
            obj_geolocation.add_attribute('city', type='text', value=shodanQueryMatch.get('location').get('city'))
            obj_geolocation.add_attribute('country', type='text', value=shodanQueryMatch.get('location').get('country_code'))
            obj_geolocation.add_attribute('latitude', type='float', value=shodanQueryMatch.get('location').get('latitude'))
            obj_geolocation.add_attribute('longitude', type='float', value=shodanQueryMatch.get('location').get('longitude'))
            obj_geolocation.add_attribute('zipcode', type='text', value=shodanQueryMatch.get('location').get('postal_code'))
            obj_geolocation.add_attribute('region', type='text', value=shodanQueryMatch.get('location').get('region_code'))
            #obj_geolocation.add_attribute('ISO6709', type='text', value=str(shodanQueryMatch.get('location').get('latitude'))+str(shodanQueryMatch.get('location').get('longitude'))+'/') # totally a lame hack

        # Add objects to event
        logger.debug('\t[to-misp] Adding obj_geolocation to event')
        event.add_object(obj_geolocation)

    # MISP http-meta object  
    if shodanQueryMatch.get('http'):
        objectExists = False  # holds if the provided http-meta object should be used or if a new object should be created
        if mispEvent and intelctlObj:  # if a mispevent and intelobject are provided, then the http-meta may already exist and be the same                          
            
            # Check if values in provided http-meta object are the same as the one which already exists in MISP
            findAll = []
            if shodanQueryMatch.get('http').get('html_hash'):
                findAll.append(('http_html_hash', str(shodanQueryMatch.get('http').get('html_hash')))) # was int but in MISP as text
            if shodanQueryMatch.get('http').get('robots_hash'):
                findAll.append(('http_robots_hash', str(shodanQueryMatch.get('http').get('robots_hash')))) # was int but in MISP as text
            if shodanQueryMatch.get('http').get('securitytxt_hash'):
                findAll.append(('http_securitytxt_hash', str(shodanQueryMatch.get('http').get('securitytxt_hash')))) # was int but in MISP as text
            if shodanQueryMatch.get('http').get('http_sitemap_hash'):
                findAll.append(('http_sitemap_hash', str(shodanQueryMatch.get('http').get('http_sitemap_hash')))) # was int but in MISP as text            
            if shodanQueryMatch.get('http').get('favicon'):
                if shodanQueryMatch.get('http').get('favicon').get('hash'):
                    findAll.append(('http_favicon_hash', str(shodanQueryMatch.get('http').get('favicon').get('hash')))) # was int but in MISP as text
            objectWithValue = misp_object_find_compare(mispEvent=event, objectName='http-metadata', attributeNameCompareList=findAll)
            if objectWithValue:
                logger.debug('\t[to-misp] Using existing http-metadata object')
                obj_httpmetadata = objectWithValue
                obj_httpmetadata.add_reference(obj_shodanreport, "derived-from")
                #obj_httpmetadata.add_reference(obj_domainip, "produced")
                objectExists = True
            else:
                 objectExists = False

        if objectExists == False: # create new http-meta  object
            logger.debug('\t[to-misp] Creating new http-metadata object') 
            obj_httpmetadata = MISPObject('http-metadata') # custom object
            obj_httpmetadata.add_reference(obj_shodanreport, "derived-from")
            obj_httpmetadata.add_reference(obj_domainip, "delivered-by")
            obj_httpmetadata.add_attribute('http_html_hash', type='text', value=shodanQueryMatch.get('http').get('html_hash')) # Numeric hash of the http.html property
            obj_httpmetadata.add_attribute('http_robots_hash', type='text', value=shodanQueryMatch.get('http').get('robots_hash')) # Numeric hash of the http.robots property
            obj_httpmetadata.add_attribute('http_securitytxt_hash', type='text', value=shodanQueryMatch.get('http').get('securitytxt_hash')) # Numeric hash of the http.securitytxt property
            obj_httpmetadata.add_attribute('http_sitemap_hash', type='text', value=shodanQueryMatch.get('http').get('sitemap_hash')) # Numeric hash of the http.sitemap property  
            obj_httpmetadata.add_attribute('http_title', type='text', value=shodanQueryMatch.get('http').get('title'))
            obj_httpmetadata.add_attribute('http_host', type='domain', value=shodanQueryMatch.get('http').get('host')) # http host header. This can be an IP. 
            obj_httpmetadata.add_attribute('http_html', type='text', value=shodanQueryMatch.get('http').get('html') , disable_correlation=True) # can be a lot of html. Correlation will use html hash
            obj_httpmetadata.add_attribute('http_server', type='text', value=shodanQueryMatch.get('http').get('server'))
            if shodanQueryMatch.get('http').get('favicon'):
                obj_httpmetadata.add_attribute('http_favicon_hash', type='text', value=shodanQueryMatch.get('http').get('favicon').get('hash')) # Numeric hash of the http.favicon.data property       

        # Add objects to event
        logger.debug('\t[to-misp] Adding obj_httpmetadata to event')
        event.add_object(obj_httpmetadata)
    
    # MISP ftp object
    if shodanQueryMatch.get('ftp'):
        logger.debug('\t[to-misp] Creating ftp-meta object') 
        obj_ftppmeta = MISPObject('ftp-meta')  # custom object
        obj_ftppmeta.add_reference(obj_shodanreport, "derived-from")
        obj_ftppmeta.add_attribute('ftp_features_hash', type='text', value=shodanQueryMatch.get('ftp').get('features_hash')) # Numeric hash of the list of supported ftp.features
        obj_ftppmeta.add_attribute('ftp_features_hash', type='text', value=shodanQueryMatch.get('ftp').get('anonymous')) # true if anonymous access is allowed – false otherwise
 
        # Add objects to event
        logger.debug('\t[to-misp] Adding obj_ftppmeta to event')
        event.add_object(obj_ftppmeta)
        
    # MISP x509 object and its attributes
    if shodanQueryMatch.get('ssl'):
        logger.debug('\t[to-misp] Starting x509 object')
        
        if shodanQueryMatch.get('ssl').get('cert'):   
            objectExists = False
            if mispEvent and intelctlObj:  # if a mispevent and intelobject are provided, then the x509 may already exist and be the same
                objectWithValue = misp_object_find_compare(mispEvent=event, objectName='x509', attributeNameCompareList=[('x509-fingerprint-sha256', shodanQueryMatch.get('ssl').get('cert').get('fingerprint').get('sha256'))])
                if objectWithValue:
                    logger.debug('\t[to-misp] Using existing x509 object')
                    obj_x509 = objectWithValue
                    obj_x509.add_reference(obj_shodanreport, "derived-from")
                    objectExists = True
                else:
                     objectExists = False
   
            if objectExists == False:  # Create new object
                logger.debug("\t[to-misp] Creating new x509 object")
                obj_x509 = MISPObject('x509')  # Object template https://www.misp-project.org/objects.html#_x509
                obj_x509.add_reference(obj_shodanreport, "derived-from")
                obj_x509.add_reference(obj_domainip, "delivered-by")
                obj_x509.add_attribute('x509-fingerprint-sha256', type='x509-fingerprint-sha256', value=shodanQueryMatch.get('ssl').get('cert').get('fingerprint').get('sha256'))
                obj_x509.add_attribute('x509-fingerprint-sha1', type='x509-fingerprint-sha1', value=shodanQueryMatch.get('ssl').get('cert').get('fingerprint').get('sha1'))
                obj_x509.add_attribute('x509-fingerprint-sha1', type='x509-fingerprint-sha1', value=shodanQueryMatch.get('ssl').get('cert').get('fingerprint').get('sha1'))
                obj_x509.add_attribute('serial-number', type='text', value=shodanQueryMatch.get('ssl').get('cert').get('serial'))
                obj_x509.add_attribute('validity-not-after', type='datetime', value=shodanQueryMatch.get('ssl').get('cert').get('expires'))
                obj_x509.add_attribute('validity-not-before', type='datetime', value=shodanQueryMatch.get('ssl').get('cert').get('issued'))
                obj_x509.add_attribute('first-seen', type='datetime', value=shodanQueryMatch.get('ssl').get('cert').get('issued')) # this allows the object to be plotted on the event timeline
                obj_x509.add_attribute('version', type='text', value=shodanQueryMatch.get('ssl').get('cert').get('version'), disable_correlation=True)
                obj_x509.add_attribute('signature_algorithm', type='text', value=shodanQueryMatch.get('ssl').get('cert').get('sig_alg'), disable_correlation=True)
                if shodanQueryMatch.get('ssl').get('cert').get('issuer'):
                    obj_x509.add_attribute('issuer', type='text', value=shodanQueryMatch.get('ssl').get('cert').get('issuer').get('O'))
                if shodanQueryMatch.get('ssl').get('cert').get('subject'):
                    obj_x509.add_attribute('dns_names', type='text', value=shodanQueryMatch.get('ssl').get('cert').get('subject').get('CN')) # Not sure I like this being type text. Sometimes this wil be a domainname
                for extdata in shodanQueryMatch.get('ssl').get('cert').get('extensions'): # Run through X509 extensinos and extract out the domain names and put each one into an attribute
                    if extdata['name'] == "subjectAltName":
                        subjectAltNames = re.findall('\\\\x[a-z0-9]{2}([a-z0-9\.\-]{2,255})', extdata['data']) # I think this is the only way to do it because the input values dont look complete.
                        for subjectAltName in subjectAltNames:
                            obj_x509.add_attribute('dns_names', type='hostname', value=subjectAltName)

            # Add objects to event
            logger.debug('\t[to-misp] Adding obj_x509 to event')
            event.add_object(obj_x509)

    # Add objects to event
    logger.debug('\t[to-misp] Adding obj_intelcollection to event')
    event.add_object(obj_intelcollection)

    # Add objects to event
    logger.debug('\t[to-misp] Adding obj_shodanreport to event')
    event.add_object(obj_shodanreport)

    # Add objects to event
    logger.debug('\t[to-misp] Adding obj_domainip to event')
    event.add_object(obj_domainip)

    # Push event to MISP
    if intelctlObj:  # MISPObject is passed thus need to update current MISP event and not make a new one.
        logger.info('\t[to-misp] Updating current MISP event')
        pymisp.update_event(event, pythonify=True) # Update an event on a MISP instance
    else:  # create new MISP event
        logger.info('\t[to-misp] Creating new MISP event')
        pymisp.add_event(event, pythonify=True) # Add a new event on a MISP instance

# Initiate search against Shodan API and collect results
queryResult = do_query(your_shodan_query, "search")
queryResultTotal = queryResult.get('total')
logger.info("Shodan API self reported this many results: {}".format(queryResultTotal))
logger.debug("Manual count of number results: {}".format(len(queryResult['matches'])))

# API returns data we want in matches key
queryMatches = queryResult.get('matches')
if queryMatches:  # there are matches to query
    for queryCounter, match in enumerate(queryResult.get('matches')):
        logger.info("__________________ Shodan search result: {} __________________".format(queryCounter+1))      
 
        # From the Shodan search, extract the IP address from the returned result and initiate a Shodan host history query
        resultIPstr = match.get('ip_str')  # Shodan IP address as a string
        logger.info("Conducting Shodan host history search on: {}".format(resultIPstr))
        hosthistory = do_query(resultIPstr,"host")
        logger.debug("  Shodan API self reported this many results: {}".format(queryResult.get('total')))

        for historyCounter, hostSearchResult in enumerate(hosthistory['data']): # each result returned from Shodan host history search   
            logger.info("-------======= Shodan search: {} Host history result: {} =======-------".format(queryCounter+1,historyCounter+1))          
            
            #
            # Quick check shortcut
            #
            # Check if the shodan report is already in MISP. If so, no need to do anything else. Go to next iteration.
            # Create unique ID. Check if the unique value is present in MISP. If present, the Shodan log entry is already in MISP
            # This is needed because the _shodan.id value is NOT always present in the search reults thus cannot be used as an indicator that the log has been processed
            uniqID = your_shodan_query + hostSearchResult.get('ip_str') + str(hostSearchResult.get('hash')) # + hostSearchResult.get('timestamp')
            uniqIDhash = hashlib.md5(uniqID.encode('utf-8')).hexdigest()
            logger.info("\tChecking {}".format(uniqIDhash))
            # Does this host history item already exist in any MISP Event? Use unique hash and search event
            hashCheck = pymisp.search(controller='attributes', type_attribute='text',  value=uniqIDhash)

            if len(hashCheck['Attribute']) > 1:
                logger.warning("\tThere are more than 1 unique hash {} with the Shodan query `{}` and IP `{}` in it. This shouldnt happen!".format(uniqIDhash, your_shodan_query, resultIPstr))           

            if len(hashCheck['Attribute']) > 0: # There are search results
                logger.info("\tAlready processed {}".format(uniqIDhash))
                continue # to next Shodan host history result

            # Could not make complex_query work correctly so doing it this silly way
            # Find the MISP events that have the shodan query in the event metadata
            possibleEventsbymeta = set()
            searchResultsShodanQuery = pymisp.search(eventinfo=your_shodan_query, metadata=True, pythonify=True)

            #if len(searchResultsShodanQuery) > 1: # should not happen. There should only be one match.
            #    logger.warning("\tThere are more than 1 MISP events with the Shodan query `{}` and IP `{}` in it. Will use first MISP event only.".format(your_shodan_query, resultIPstr))

            if len(searchResultsShodanQuery) >= 1: # the shodan query has been run before because MISP events are seen with it
                for myresult in searchResultsShodanQuery:
                    logger.debug("\tMISP event {} has Shodan Query".format(myresult.id))
                    possibleEventsbymeta.add(myresult.id)

                # Find the MISP event that has the IP address in question
                searchResultsIP = pymisp.search(controller='attributes', value=resultIPstr, pythonify=True)
                possibleEventsbyIP = set()

                if len(searchResultsIP) > 0: # There are results for the IP address in a MISP event
                    for myresult in searchResultsIP: # put all MISP event IDs into a set
                        logger.debug("\tMISP event {} has IP address".format(myresult.event_id))
                        possibleEventsbyIP.add(myresult.event_id)

                    for mispeventID in possibleEventsbyIP: # check if MISP event ID for IP matches that of Shodan query found in MISP event metadata
                        if mispeventID in possibleEventsbymeta:
                            logger.info("\tUsing {} as MISP event for processing".format(mispeventID))
                            searchresults = pymisp.get_event(mispeventID) 

                            #
                            # Use found MISP Event 
                            #
                            # This is the MISP event where all Shodan history results for the  will go
                            mispEventParent = pymisp.get_event(mispeventID, pythonify=True)
                            logger.debug("\tMISP Event for this Shodan query and IP is {}".format(mispeventID))
                            logger.debug("\tProcessing {}".format(uniqIDhash))
                            # Get intel-collection MISPobject
                            intelCollectionObj = mispEventParent.get_objects_by_name('intel-collection')[0]  # dont think there would ever be more than 1 hit here
                            shodan_to_misp_event(shodanQueryMatch=hostSearchResult, shodanQuery=your_shodan_query, shodanQueryPurpose=search_reason, intelctlObj=intelCollectionObj, mispEvent=mispEventParent)      

                        else:  # shodan query found BUT the IP is not found so a new MISP event is needed
                            #
                            # Create a MISP Event 
                            #
                            logger.info("\tCreating MISP event for Shodan query `{}` and IP `{}` in it".format(your_shodan_query, resultIPstr))
                            # Put shodan result into MISP event 
                            shodan_to_misp_event(shodanQueryMatch=hostSearchResult, shodanQuery=your_shodan_query, shodanQueryPurpose=search_reason, intelctlObj=None, mispEvent=None)   
                            continue  # to next Shodan host history result

                else:  # shodan query found BUT the IP is not found so a new MISP event is needed
                    #
                    # Create a MISP Event 
                    #
                    logger.info("\tCreating MISP event for Shodan query `{}` and IP `{}` in it".format(your_shodan_query, resultIPstr))
                    # Put shodan result into MISP event 
                    shodan_to_misp_event(shodanQueryMatch=hostSearchResult, shodanQuery=your_shodan_query, shodanQueryPurpose=search_reason, intelctlObj=None, mispEvent=None)   
                    continue  # to next Shodan host history result


            else:  # Shodan query not in MISP so create it
                #
                # Create a MISP Event 
                #
                logger.info("\tCreating MISP event for Shodan query `{}` and IP `{}` in it".format(your_shodan_query, resultIPstr))
                # Put shodan result into MISP event 
                shodan_to_misp_event(shodanQueryMatch=hostSearchResult, shodanQuery=your_shodan_query, shodanQueryPurpose=search_reason, intelctlObj=None, mispEvent=None)   
                continue  # to next Shodan host history result


else: # there are not matches to query
    logger.info("No query results")


# Example: Searching MISP for Value in Specific Attribute
Searches an `ip-dst` attribute type with the given value

In [8]:
searchresults = pymisp.search(controller='attributes',type_attribute='ip-dst',  value='45.66.41.15', include_event_uuid=True)
if len(searchresults['Attribute']) == 0:
    print('No Matches')
for result in searchresults['Attribute']:
    print("MATCH! Type {} with value: {} and UUID: {} | Match is in objectID: {} | Match in EventID {} with EventUUID: {}".format(result['type'], result['value'], result['uuid'], result['Object']['id'], result['event_id'], result['event_uuid']))


MATCH! Type ip-dst with value: 45.66.41.15 and UUID: 7fbfed10-6781-45c8-aa54-ca7035ace001 | Match is in objectID: 7407 | Match in EventID 850 with EventUUID: ce0bea5d-2dec-4797-a8c2-486e68516c69


# Example: Searching MISP for UUID
Given a UUID, this code searches MISP and will provide what it is used for (e.g. event, attribute)

In [9]:
# UUID to search for
myuuid = 'ce0bea5d-2dec-4797-a8c2-486e68516c69'

searchresults = pymisp.search(uuid=myuuid, include_event_uuid=True)
print("Total results: {}".format(len(searchresults)))

for item in searchresults:
    if item['Event']['uuid'] == myuuid:
        print('FOUND as event UUID')
    if item['Event']['Org']['uuid'] == myuuid:
        print('FOUND as org UUID')
    if item['Event']['Orgc']['uuid'] == myuuid:
        print('FOUND as orgc UUID')

    for relatedEvent in item['Event']['RelatedEvent']:
        #print(json.dumps(relatedEvent, indent=2))
        if relatedEvent['Event']['uuid'] == myuuid:
            print('FOUND as event UUID')
        if relatedEvent['Event']['Org']['uuid'] == myuuid:
            print('FOUND as org UUID')
        if relatedEvent['Event']['Orgc']['uuid'] == myuuid:
            print('FOUND as orgc UUID')

    for eventObject in item['Event']['Object']: 
        if eventObject['uuid'] == myuuid:
            print('FOUND as object UUID')
        for objectAttribute in eventObject['Attribute']:
            if objectAttribute['uuid'] == myuuid:
                print('FOUND as attribute UUID')
        #print(json.dumps(eventObject, indent=2))

Total results: 1
FOUND as event UUID


# Example: Searching MISP for String Timestamps
A timestamp from Shodan that comes in as `"timestamp": "2020-06-29T12:59:20.821901"` and entered in as a MISP datetime value, can be searched as `2020-06-29T12:59:20.821901+0000`

In [10]:
searchresults = pymisp.search(controller='attributes', value='2020-06-22T08:53:46.361727+0000')
print("Number of results: {}".format(len(searchresults['Attribute'])))
for result in searchresults['Attribute']:
    print(result['event_id'],result['type'], result['value'])
    print(json.dumps(result, indent=2))
    


Number of results: 1
850 datetime 2020-06-22T08:53:46.361727+0000
{
  "id": "46294",
  "event_id": "850",
  "object_id": "7427",
  "object_relation": "first-seen",
  "category": "Other",
  "type": "datetime",
  "to_ids": false,
  "uuid": "ead6b034-5f8d-4f43-b6d3-b215e1038d7e",
  "timestamp": "1596894123",
  "distribution": "5",
  "sharing_group_id": "0",
  "comment": "",
  "deleted": false,
  "disable_correlation": false,
  "first_seen": null,
  "last_seen": null,
  "value": "2020-06-22T08:53:46.361727+0000",
  "Event": {
    "org_id": "1",
    "distribution": "0",
    "id": "850",
    "info": "VNC config",
    "orgc_id": "1",
    "uuid": "ce0bea5d-2dec-4797-a8c2-486e68516c69"
  },
  "Object": {
    "id": "7427",
    "distribution": "5",
    "sharing_group_id": "0"
  }
}


# Example: Searching MISP Complex Query
The complex query does not seem to work correctly OR I'm not using it right. The example search should provide NO results yet it does.

In [11]:
complex_query = {'AND': ['VNC config', 'random_should_not_match']} # type dict
searchresults = pymisp.search(value=complex_query)
for result in searchresults:
    print(result['Event']['id'])

849
850


# Example: Get MISP UUID Attribute

In [17]:
uuid_results = pymisp.get_attribute('7fbfed10-6781-45c8-aa54-ca7035ace001', pythonify=True)
print(uuid_results.value)

45.66.41.15


# Example: Looking at a specific MISP event
This first creates a MISPobject for the specified event. Then it outputs the first geolcation object as JSON.

In [22]:
my_parent = pymisp.get_event(850, pythonify=True)
my_parent.get_objects_by_name('geolocation')[0].to_json(['Attribute'])

'{"Attribute": [{"category": "Other", "comment": "", "deleted": false, "disable_correlation": false, "distribution": "5", "event_id": "850", "id": "46157", "object_id": "7404", "object_relation": "city", "sharing_group_id": "0", "timestamp": "1596894113", "to_ids": false, "type": "text", "uuid": "e912bed8-6b2a-492e-86da-02d6d904deef", "value": "Athens"}, {"category": "Other", "comment": "", "deleted": false, "disable_correlation": false, "distribution": "5", "event_id": "850", "id": "46158", "object_id": "7404", "object_relation": "country", "sharing_group_id": "0", "timestamp": "1596894113", "to_ids": false, "type": "text", "uuid": "7d3695e8-6896-489c-ae91-d9765b20f770", "value": "GR"}, {"category": "Other", "comment": "", "deleted": false, "disable_correlation": true, "distribution": "5", "event_id": "850", "id": "46159", "object_id": "7404", "object_relation": "latitude", "sharing_group_id": "0", "timestamp": "1596894113", "to_ids": false, "type": "float", "uuid": "ba10d367-842f-409